# Multi Document Agents Pack

This LlamaPack provides an example of our multi-document agents.

In [ ]:
!pip install llama-index llama-hub

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Setup Data

In [ ]:
from pathlib import Path
import requests
from llama_index.core import SimpleDirectoryReader

wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    docs = SimpleDirectoryReader(input_files=[f"data/{wiki_title}.txt"]).load_data()
    desc = f"This is a Wikipedia article about the city of {wiki_title}"
    city_docs[wiki_title] = (wiki_title, desc, docs[0])

### Download and Initialize Pack

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

MultiDocumentAgentsPack = download_llama_pack(
    "MultiDocumentAgentsPack",
    "./multi_doc_agents_pack",
    # leave the below commented out (was for testing purposes)
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llama_packs/llama_hub",
)

In [ ]:
city_tups = list(city_docs.values())
multi_doc_agents_pack = MultiDocumentAgentsPack(
    [t[2] for t in city_tups], [t[0] for t in city_tups], [t[1] for t in city_tups]
)

### Run Pack

In [ ]:
# this will run the full pack
response = multi_doc_agents_pack.run(
    "Tell me the demographics of Houston, and then compare with the demographics of Chicago"
)

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_Houston with args: {
  "input": "demographics"
}
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "demographics"
}
Got output: Houston has a population of 2,304,580 according to the 2020 U.S. census. In 2017, the estimated population was 2,312,717, and in 2018 it was 2,325,502. The city has a diverse demographic makeup, with a significant number of undocumented immigrants residing in the Houston area, comprising nearly 9% of the city's metropolitan population in 2017. The age distribution in Houston includes a significant number of individuals under 15 and between the ages of 20 to 34. The median age of the city is 33.4. The city has a mix of homeowners and renters, with an estimated 42.3% of Houstonians owning housing units. The median household income in 2019 was $52,338, and 20.1% of Houstonians lived at or below the poverty line.

STARTING 

In [ ]:
print(str(response))

During his time in YC, the author worked with a new batch of startups every six months. He engaged with the problems faced by these startups and worked hard to help them. He mentioned that there were parts of the job he didn't like, such as disputes between cofounders and dealing with people who maltreated the startups. However, he worked hard even at the parts he didn't like. The author also mentioned that he wanted YC to be successful, so he worked very hard, setting an example for others.


In [ ]:
len(response.source_nodes)

2

### Inspect Modules

In [ ]:
modules = multi_doc_agents_pack.get_modules()
display(modules)

{'vector_retriever': <llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever at 0x2bc0e76d0>,
 'fusion_retriever': <llama_index.core.retrievers.fusion_retriever.QueryFusionRetriever at 0x2bc0e7eb0>,
 'query_engine': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x2891c5990>}